In [184]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import (
    Ridge, Lasso, LinearRegression, ElasticNet
)
from sklearn.ensemble import (
    BaggingRegressor, RandomForestRegressor, 
    AdaBoostRegressor, GradientBoostingRegressor
)
from sklearn.svm import SVR

os.chdir('/Users/mathewkatz/Desktop/CunySPS/DATA698')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

In [185]:
newdf = pd.read_csv('but2020.csv')
twenty = pd.read_csv('2020.csv')
stats = pd.concat([newdf, twenty], axis=0)
stats


,"last_name, first_name",player_id,year,player_age,p_game,p_formatted_ip,pa,ab,hit,single,...,breaking_avg_break,breaking_range_speed,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break_x,offspeed_avg_break_z,offspeed_avg_break,offspeed_range_speed,Unnamed: 249
0,"Colon, Bartolo",112526,2015,42,33,194.2,815,771,217,149,...,6.5,1.8,7.4,82.6,1727.0,-13.2,NaN,16.4,2.4,NaN
1,"Burnett, A.J.",150359,2015,38,26,164.0,699,633,174,134,...,12.8,1.5,8.8,86.3,1678.0,-10.8,NaN,12.0,1.8,NaN
2,"Hudson, Tim",218596,2015,39,24,123.2,525,476,134,103,...,16.8,1.6,11.2,80.9,1369.0,-9.1,NaN,10.3,1.4,NaN
3,"Benoit, Joaquin",276542,2015,37,67,65.1,254,226,36,19,...,6.5,1.8,34.4,84.7,1434.0,-12.4,NaN,13.1,1.4,NaN
4,"Buehrle, Mark",279824,2015,36,32,198.2,827,768,214,140,...,11.9,2.0,21.1,78.7,1625.0,14.1,NaN,16.2,1.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Bumgarner, Madison",518516,2020,30,9,41.2,190,170,47,22,...,11.5,1.4,1.3,83.1,1684.0,11.8,-31.6,14.5,1.5,NaN
97,"Montgomery, Jordan",656756,2020,27,10,44.0,193,181,48,31,...,3.7,1.6,25.6,83.2,1756.0,9.6,-24.9,17.5,1.0,NaN
98,"McCullers Jr., Lance",621121,2020,26,11,55.0,227,200,44,19,...,16.1,1.8,18.4,86.3,1766.0,-16.6,-35.2,17.0,1.7,NaN
99,"Manaea, Sean",640455,2020,28,11,54.0,222,211,57,39,...,7.1,1.9,28.2,82.5,1102.0,7.1,-37.9,7.7,1.7,NaN


In [186]:
def add_next_year_era(df):
    if 'player_id' in df.columns:
        player_counts = df['player_id'].value_counts()
        filtered_df = df[df['player_id'].isin(player_counts[player_counts > 1].index)]
        if not filtered_df.empty:
            filtered_df = filtered_df.sort_values(by=['player_id', 'year'])
            filtered_df['NextYrERA'] = float('nan')
            filtered_df.set_index(['player_id', 'year'], inplace=True)
            for index, row in filtered_df.iterrows():
                player_id, current_year = index
                if (player_id, current_year + 1) in filtered_df.index:
                    next_year_era = filtered_df.loc[(player_id, current_year + 1), 'p_era']
                    filtered_df.at[index, 'NextYrERA'] = next_year_era
                else:
                    filtered_df.at[index, 'NextYrERA'] = float('nan')
            filtered_df.reset_index(inplace=True)
            return filtered_df
        else:
            print("No duplicate player_id values in the DataFrame.")
            return df
    else:
        print("Invalid DataFrame")
        return None

In [187]:
stats = add_next_year_era(stats)
stats

,player_id,year,"last_name, first_name",player_age,p_game,p_formatted_ip,pa,ab,hit,single,...,breaking_range_speed,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break_x,offspeed_avg_break_z,offspeed_avg_break,offspeed_range_speed,Unnamed: 249,NextYrERA
0,112526,2015,"Colon, Bartolo",42,33,194.2,815,771,217,149,...,1.8,7.4,82.6,1727.0,-13.2,NaN,16.4,2.4,NaN,3.43
1,112526,2016,"Colon, Bartolo",43,34,191.2,791,745,200,134,...,1.9,4.3,81.4,1683.0,-13.2,-32.0,16.2,1.5,NaN,6.48
2,112526,2017,"Colon, Bartolo",44,28,143.0,648,603,192,110,...,1.7,9.7,81.3,1678.0,-14.3,-33.6,18.1,1.3,NaN,5.78
3,112526,2018,"Colon, Bartolo",45,28,146.1,628,591,172,95,...,1.9,11.1,80.8,1657.0,-13.8,-36.1,16.1,1.4,NaN,NaN
4,282332,2015,"Sabathia, CC",34,29,167.1,726,659,188,134,...,1.5,14.0,83.9,1950.0,11.1,NaN,15.0,1.2,NaN,3.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,680686,2021,"Gray, Josiah",23,14,70.2,307,265,63,32,...,1.5,3.3,89.2,2218.0,-14.2,-22.2,18.9,1.4,NaN,5.02
1983,680686,2022,"Gray, Josiah",24,28,148.2,649,569,136,74,...,2.1,2.8,88.1,2056.0,-16.2,-25.3,19.2,1.4,NaN,3.91
1984,680686,2023,"Gray, Josiah",25,30,159.0,698,605,152,100,...,2.4,1.6,88.6,1617.0,-13.4,-26.1,16.1,1.3,NaN,NaN
1985,680694,2022,"Bradish, Kyle",25,23,117.2,509,449,119,84,...,2.6,8.5,89.9,1427.0,-12.2,-24.3,15.7,1.7,NaN,2.83


In [188]:
player = stats[stats['last_name, first_name'] == 'Syndergaard, Noah']
player = player.sort_values(by='year')
selected_columns = ['last_name, first_name', 'player_id', 'year', 'p_era', 'NextYrERA']
player[selected_columns]

,"last_name, first_name",player_id,year,p_era,NextYrERA
1115,"Syndergaard, Noah",592789,2015,3.24,2.60
1116,"Syndergaard, Noah",592789,2016,2.60,NaN
1117,"Syndergaard, Noah",592789,2018,3.03,4.28
1118,"Syndergaard, Noah",592789,2019,4.28,NaN
1119,"Syndergaard, Noah",592789,2022,3.94,6.50
1120,"Syndergaard, Noah",592789,2023,6.50,NaN


In [189]:
player2 = stats[stats['last_name, first_name'] == 'Greinke, Zack']
player2 = player2.sort_values(by='year')
selected_columns = ['last_name, first_name', 'player_id', 'year', 'p_era', 'NextYrERA']
player2[selected_columns]

,"last_name, first_name",player_id,year,p_era,NextYrERA
33,"Greinke, Zack",425844,2015,1.66,4.37
34,"Greinke, Zack",425844,2016,4.37,3.20
35,"Greinke, Zack",425844,2017,3.20,3.21
36,"Greinke, Zack",425844,2018,3.21,2.93
37,"Greinke, Zack",425844,2019,2.93,4.03
38,"Greinke, Zack",425844,2020,4.03,4.16
39,"Greinke, Zack",425844,2021,4.16,3.68
40,"Greinke, Zack",425844,2022,3.68,5.06
41,"Greinke, Zack",425844,2023,5.06,NaN


In [190]:
player3 = stats[stats['last_name, first_name'] == 'deGrom, Jacob']
player3 = player3.sort_values(by='year')
selected_columns = ['last_name, first_name', 'player_id', 'year', 'p_era', 'NextYrERA']
player3[selected_columns]

,"last_name, first_name",player_id,year,p_era,NextYrERA
1183,"deGrom, Jacob",594798,2015,2.54,3.04
1184,"deGrom, Jacob",594798,2016,3.04,3.53
1185,"deGrom, Jacob",594798,2017,3.53,1.70
1186,"deGrom, Jacob",594798,2018,1.70,2.43
1187,"deGrom, Jacob",594798,2019,2.43,2.38
1188,"deGrom, Jacob",594798,2020,2.38,1.08
1189,"deGrom, Jacob",594798,2021,1.08,NaN


In [191]:
player4 = stats[stats['last_name, first_name'] == 'Corbin, Patrick']
player4 = player4.sort_values(by='year')
selected_columns = ['last_name, first_name', 'player_id', 'year', 'p_era', 'NextYrERA']
player4[selected_columns]

,"last_name, first_name",player_id,year,p_era,NextYrERA
929,"Corbin, Patrick",571578,2015,3.60,5.15
930,"Corbin, Patrick",571578,2016,5.15,4.03
931,"Corbin, Patrick",571578,2017,4.03,3.15
932,"Corbin, Patrick",571578,2018,3.15,3.25
933,"Corbin, Patrick",571578,2019,3.25,4.66
934,"Corbin, Patrick",571578,2020,4.66,5.82
935,"Corbin, Patrick",571578,2021,5.82,6.31
936,"Corbin, Patrick",571578,2022,6.31,5.20
937,"Corbin, Patrick",571578,2023,5.20,NaN


In [192]:
player5 = stats[stats['last_name, first_name'] == 'Alcantara, Sandy']
player5 = player5.sort_values(by='year')
selected_columns = ['last_name, first_name', 'player_id', 'year', 'p_era', 'NextYrERA']
player5[selected_columns]

,"last_name, first_name",player_id,year,p_era,NextYrERA
1737,"Alcantara, Sandy",645261,2019,3.88,NaN
1738,"Alcantara, Sandy",645261,2021,3.19,2.28
1739,"Alcantara, Sandy",645261,2022,2.28,4.14
1740,"Alcantara, Sandy",645261,2023,4.14,NaN


In [193]:
stats

,player_id,year,"last_name, first_name",player_age,p_game,p_formatted_ip,pa,ab,hit,single,...,breaking_range_speed,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break_x,offspeed_avg_break_z,offspeed_avg_break,offspeed_range_speed,Unnamed: 249,NextYrERA
0,112526,2015,"Colon, Bartolo",42,33,194.2,815,771,217,149,...,1.8,7.4,82.6,1727.0,-13.2,NaN,16.4,2.4,NaN,3.43
1,112526,2016,"Colon, Bartolo",43,34,191.2,791,745,200,134,...,1.9,4.3,81.4,1683.0,-13.2,-32.0,16.2,1.5,NaN,6.48
2,112526,2017,"Colon, Bartolo",44,28,143.0,648,603,192,110,...,1.7,9.7,81.3,1678.0,-14.3,-33.6,18.1,1.3,NaN,5.78
3,112526,2018,"Colon, Bartolo",45,28,146.1,628,591,172,95,...,1.9,11.1,80.8,1657.0,-13.8,-36.1,16.1,1.4,NaN,NaN
4,282332,2015,"Sabathia, CC",34,29,167.1,726,659,188,134,...,1.5,14.0,83.9,1950.0,11.1,NaN,15.0,1.2,NaN,3.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,680686,2021,"Gray, Josiah",23,14,70.2,307,265,63,32,...,1.5,3.3,89.2,2218.0,-14.2,-22.2,18.9,1.4,NaN,5.02
1983,680686,2022,"Gray, Josiah",24,28,148.2,649,569,136,74,...,2.1,2.8,88.1,2056.0,-16.2,-25.3,19.2,1.4,NaN,3.91
1984,680686,2023,"Gray, Josiah",25,30,159.0,698,605,152,100,...,2.4,1.6,88.6,1617.0,-13.4,-26.1,16.1,1.3,NaN,NaN
1985,680694,2022,"Bradish, Kyle",25,23,117.2,509,449,119,84,...,2.6,8.5,89.9,1427.0,-12.2,-24.3,15.7,1.7,NaN,2.83


In [194]:
null_count = stats.isnull().sum()

complete_cols = list(stats.columns[null_count == 0])

stats = stats[complete_cols + ["NextYrERA"]].copy()

stats

,player_id,year,"last_name, first_name",player_age,p_game,p_formatted_ip,pa,ab,hit,single,...,popups,pitch_hand,n,n_fastball_formatted,fastball_avg_speed,fastball_avg_spin,fastball_avg_break_x,fastball_avg_break,fastball_range_speed,NextYrERA
0,112526,2015,"Colon, Bartolo",42,33,194.2,815,771,217,149,...,37,R,2683,82.6,88.9,2161,-12.4,17.4,2.4,3.43
1,112526,2016,"Colon, Bartolo",43,34,191.2,791,745,200,134,...,40,R,2843,89.4,88.4,2148,-12.8,17.4,2.3,6.48
2,112526,2017,"Colon, Bartolo",44,28,143.0,648,603,192,110,...,34,R,2280,83.0,87.8,2124,-14.7,20.3,2.3,5.78
3,112526,2018,"Colon, Bartolo",45,28,146.1,628,591,172,95,...,29,R,2228,81.5,87.3,2171,-14.6,18.6,2.1,NaN
4,282332,2015,"Sabathia, CC",34,29,167.1,726,659,188,134,...,32,L,2693,63.6,90.2,2075,12.2,18.1,2.2,3.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,680686,2021,"Gray, Josiah",23,14,70.2,307,265,63,32,...,29,R,1213,51.6,94.6,2254,-9.5,20.2,1.1,5.02
1983,680686,2022,"Gray, Josiah",24,28,148.2,649,569,136,74,...,45,R,2596,43.4,94.3,2219,-10.4,20.4,1.5,3.91
1984,680686,2023,"Gray, Josiah",25,30,159.0,698,605,152,100,...,46,R,2812,52.0,91.8,2083,-8.8,15.9,2.7,NaN
1985,680694,2022,"Bradish, Kyle",25,23,117.2,509,449,119,84,...,15,R,1993,48.8,94.7,2292,0.1,16.6,1.5,2.83


In [195]:
stats.dtypes[stats.dtypes == "object"]


last_name, first_name    object
pitch_hand               object
dtype: object

In [196]:
del stats['pitch_hand']

In [197]:
stats = stats.dropna().copy()

In [198]:
stats

,player_id,year,"last_name, first_name",player_age,p_game,p_formatted_ip,pa,ab,hit,single,...,popups_percent,popups,n,n_fastball_formatted,fastball_avg_speed,fastball_avg_spin,fastball_avg_break_x,fastball_avg_break,fastball_range_speed,NextYrERA
0,112526,2015,"Colon, Bartolo",42,33,194.2,815,771,217,149,...,5.7,37,2683,82.6,88.9,2161,-12.4,17.4,2.4,3.43
1,112526,2016,"Colon, Bartolo",43,34,191.2,791,745,200,134,...,6.4,40,2843,89.4,88.4,2148,-12.8,17.4,2.3,6.48
2,112526,2017,"Colon, Bartolo",44,28,143.0,648,603,192,110,...,6.5,34,2280,83.0,87.8,2124,-14.7,20.3,2.3,5.78
4,282332,2015,"Sabathia, CC",34,29,167.1,726,659,188,134,...,6.0,32,2693,63.6,90.2,2075,12.2,18.1,2.2,3.91
5,282332,2016,"Sabathia, CC",35,30,179.2,768,687,172,122,...,8.5,46,2908,71.4,89.3,2104,7.7,14.3,3.0,3.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,677651,2021,"Garcia, Luis",24,30,155.1,633,573,133,81,...,8.5,35,2502,67.1,90.9,2362,-1.5,15.7,4.1,3.72
1980,678394,2022,"Bello, Brayan",23,13,57.1,268,238,75,55,...,2.7,5,997,53.9,96.5,2150,-14.5,17.4,1.1,4.24
1982,680686,2021,"Gray, Josiah",23,14,70.2,307,265,63,32,...,14.8,29,1213,51.6,94.6,2254,-9.5,20.2,1.1,5.02
1983,680686,2022,"Gray, Josiah",24,28,148.2,649,569,136,74,...,10.7,45,2596,43.4,94.3,2219,-10.4,20.4,1.5,3.91


In [199]:
# Create a Ridge regression model with regularization parameter alpha set to 1
ridge = Ridge(alpha=1)
# Create a TimeSeriesSplit object with 3 splits for time series cross-validation
split = TimeSeriesSplit(n_splits=3)
# Create a SequentialFeatureSelector (SFS) object
# This is used for feature selection in a forward manner (adding features)
# It uses the Ridge regression model for evaluation
# It aims to select 20 features during the selection process
# The cross-validation strategy is based on the TimeSeriesSplit with 3 splits
# The feature selection process will be parallelized using 8 jobs
sfs = SequentialFeatureSelector(ridge, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=8
                               )

In [200]:
removed_columns = ["NextYrERA", "player_id", "year", "last_name, first_name"]
selected_columns = stats.columns[~stats.columns.isin(removed_columns)]

In [201]:
scaler = MinMaxScaler()
stats.loc[:,selected_columns] = scaler.fit_transform(stats[selected_columns])

In [202]:
stats

,player_id,year,"last_name, first_name",player_age,p_game,p_formatted_ip,pa,ab,hit,single,...,popups_percent,popups,n,n_fastball_formatted,fastball_avg_speed,fastball_avg_spin,fastball_avg_break_x,fastball_avg_break,fastball_range_speed,NextYrERA
0,112526,2015,"Colon, Bartolo",0.916667,0.337838,0.801047,0.821288,0.848739,0.891509,0.921986,...,0.296512,0.423529,0.663193,0.814898,0.356757,0.347869,0.137741,0.581699,0.355556,3.43
1,112526,2016,"Colon, Bartolo",0.958333,0.351351,0.785340,0.789750,0.812325,0.811321,0.815603,...,0.337209,0.458824,0.716975,0.891648,0.329730,0.334985,0.126722,0.581699,0.333333,6.48
2,112526,2017,"Colon, Bartolo",1.000000,0.270270,0.532984,0.601840,0.613445,0.773585,0.645390,...,0.343023,0.388235,0.527731,0.819413,0.297297,0.311199,0.074380,0.771242,0.333333,5.78
4,282332,2015,"Sabathia, CC",0.583333,0.283784,0.659162,0.704336,0.691877,0.754717,0.815603,...,0.313953,0.364706,0.666555,0.600451,0.427027,0.262636,0.815427,0.627451,0.311111,3.91
5,282332,2016,"Sabathia, CC",0.625000,0.297297,0.722513,0.759527,0.731092,0.679245,0.730496,...,0.459302,0.529412,0.738824,0.688488,0.378378,0.291378,0.691460,0.379085,0.488889,3.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,677651,2021,"Garcia, Luis",0.166667,0.297297,0.596335,0.582129,0.571429,0.495283,0.439716,...,0.459302,0.400000,0.602353,0.639955,0.464865,0.547076,0.438017,0.470588,0.733333,3.72
1980,678394,2022,"Bello, Brayan",0.125000,0.067568,0.083246,0.102497,0.102241,0.221698,0.255319,...,0.122093,0.047059,0.096471,0.490971,0.767568,0.336967,0.079890,0.581699,0.066667,4.24
1982,680686,2021,"Gray, Josiah",0.125000,0.081081,0.151832,0.153745,0.140056,0.165094,0.092199,...,0.825581,0.329412,0.169076,0.465011,0.664865,0.440040,0.217631,0.764706,0.066667,5.02
1983,680686,2022,"Gray, Josiah",0.166667,0.270270,0.560209,0.603154,0.565826,0.509434,0.390071,...,0.587209,0.517647,0.633950,0.372460,0.648649,0.405352,0.192837,0.777778,0.155556,3.91


In [203]:
sfs.fit(stats[selected_columns], stats["NextYrERA"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=8)

In [204]:
predictors = list(selected_columns[sfs.get_support()])

In [205]:
predictors

['k_percent',
 'p_loss',
 'p_caught_stealing_2b',
 'p_game_finished',
 'p_gnd_into_tp',
 'p_gnd_rule_double',
 'p_hit_by_pitch',
 'p_missed_bunt',
 'p_total_pickoff',
 'p_total_pickoff_error',
 'p_inh_runner_scored',
 'xslgdiff',
 'launch_angle_avg',
 'sweet_spot_percent',
 'avg_best_speed',
 'out_zone_percent',
 'pitch_count_fastball',
 'opposite_percent',
 'popups_percent',
 'fastball_avg_spin']

In [206]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []
    
    years = sorted(data["year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["year"] < current_year]
        test = data[data["year"] == current_year]
        
        model.fit(train[predictors], train["NextYrERA"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["NextYrERA"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [207]:
predictions = backtest(stats, ridge, predictors)


In [208]:
predictions.shape


(414, 2)

In [209]:
mean_squared_error(predictions["actual"], predictions["prediction"])


1.0261073137949286

In [210]:
stats['NextYrERA'].describe()

count    1207.000000
mean        4.115178
std         1.167635
min         0.540000
25%         3.315000
50%         4.040000
75%         4.810000
max         9.580000
Name: NextYrERA, dtype: float64

In [211]:
1.0261073137949286 ** .5

1.0129695522546216

In [212]:
pd.Series(ridge.coef_, index=predictors).sort_values()


k_percent               -2.728464
p_inh_runner_scored     -0.529145
fastball_avg_spin       -0.399682
p_game_finished         -0.376590
p_hit_by_pitch          -0.367978
opposite_percent        -0.359540
pitch_count_fastball    -0.327951
p_total_pickoff         -0.300751
p_gnd_into_tp           -0.276852
p_missed_bunt           -0.242695
xslgdiff                -0.236935
p_gnd_rule_double       -0.171801
launch_angle_avg         0.006963
p_loss                   0.291248
p_total_pickoff_error    0.327689
out_zone_percent         0.417621
p_caught_stealing_2b     0.438592
sweet_spot_percent       0.696350
avg_best_speed           0.784525
popups_percent           0.904369
dtype: float64

In [213]:
diff = predictions["actual"] - predictions["prediction"]
merged = predictions.merge(stats, left_index=True, right_index=True)
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()
merged[["player_id", "year", "last_name, first_name", "p_era", "NextYrERA", "diff"]].sort_values(["diff"])

/var/folders/l1/809gkysd54b3ppb9whn3tcp00000gn/T/ipykernel_41527/820178338.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()


,player_id,year,"last_name, first_name",p_era,NextYrERA,diff
1913,669022,2022,"Gore, MacKenzie",0.510288,4.42,0.000272
1143,592866,2020,"Williams, Trevor",0.740741,4.35,0.001062
1782,656756,2020,"Montgomery, Jordan",0.593964,3.83,0.002024
386,477132,2020,"Kershaw, Clayton",0.189300,3.55,0.002807
1980,678394,2022,"Bello, Brayan",0.539095,4.24,0.007241
...,...,...,...,...,...,...
31,425794,2022,"Wainwright, Adam",0.401920,7.40,2.914717
714,542914,2021,"Bass, Anthony",0.417010,1.54,2.973564
1962,672578,2021,"Hernandez, Carlos",0.397805,7.39,3.027712
1375,607074,2022,"Rodon, Carlos",0.288066,6.85,3.364045


In [ ]:
plt.figure(figsize=(10, 30))

correlation_matrix = stats.corr()[['NextYrERA']].sort_values('NextYrERA', ascending=False)

sns.heatmap(correlation_matrix, annot=True, cmap='RdBu', vmin=-1, vmax=1)

plt.title('Correlation with NextYrERA')
plt.xlabel('NextYrERA')
plt.ylabel('Features')

plt.show()

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')

f, ax = plt.subplots(figsize=(8, 7))
sns.histplot(stats['NextYrERA'], color="b", kde=True)

ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="NextYrERA")
ax.set(title="NextYrERA distribution")

sns.despine(trim=True, left=True)
plt.show()

In [ ]:
print("Skewness: %f" % stats['NextYrERA'].skew())
print("Kurtosis: %f" % stats['NextYrERA'].kurt())

The distribution of NextYrERA exhibits a slightly positively skew and a close to a normal distribution in terms of kurtosis. To address this skewness, we can apply a transformation to enhance the symmetry of the distribution.

In [ ]:
pt = PowerTransformer()

stats['NextYrERA'] = pt.fit_transform(stats['NextYrERA'].values.reshape(-1, 1))

In [ ]:
print("Skewness: %f" % stats['NextYrERA'].skew())
print("Kurtosis: %f" % stats['NextYrERA'].kurt())

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')

f, ax = plt.subplots(figsize=(8, 7))
sns.histplot(stats['NextYrERA'], color="b", kde=True)

ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="NextYrERA")
ax.set(title="NextYrERA distribution")

sns.despine(trim=True, left=True)
plt.show()

In [ ]:
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100, 2)})
    
    return dict_x

missing = percent_missing(stats)

filtered_columns = [col for col, percentage in missing.items() if percentage == 0.0]
stats = stats[filtered_columns]
stats

In [ ]:
numeric_feats = stats.dtypes[stats.dtypes != "object"].index
skewed_feats = stats[numeric_feats].apply(lambda x: skew(x.dropna())) 
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
stats.loc[:, skewed_feats] = np.log1p(stats.loc[:, skewed_feats])

In [ ]:
correlation_matrix = stats.corr()
target_correlation = correlation_matrix['NextYrERA'].abs()
threshold = 0.2
selected_features = target_correlation[target_correlation > threshold].index
selected_features

In [ ]:
index_list = selected_features.tolist()
index_list

In [ ]:
def print_nan_counts(dataframe, columns):
    columns_to_remove = []
    
    for column in columns:
        nan_count = dataframe[column].isna().sum()
        print(f"Number of NaN values in '{column}': {nan_count}")
        
        if nan_count > 1:
            columns_to_remove.append(column)
    
    if columns_to_remove:
        dataframe.drop(columns=columns_to_remove, inplace=True)
        print(f"Columns removed due to NaN values: {columns_to_remove}")
        
        columns = [col for col in columns if col not in columns_to_remove]

    return columns

index_list = print_nan_counts(stats, index_list)

In [ ]:
index_list.remove('NextYrERA')

In [ ]:
X = stats[index_list]
y = stats['NextYrERA']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)
ss = StandardScaler()
ss.fit(X_train) 

In [ ]:
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
def rmse_and_r2(regressor, X_train_sc, y_train, X_test_sc, y_test, scaler=StandardScaler()):
    steps = [
        ('scaler', scaler),
        ('regressor', regressor())
    ]
    pipe = Pipeline(steps=steps)
    model = pipe.fit(X_train_sc, y_train)
    train_pred = model.predict(X_train_sc)
    test_pred = model.predict(X_test_sc)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    return train_rmse, test_rmse, train_r2, test_r2

In [ ]:
regressors = [LinearRegression, KNeighborsRegressor, DecisionTreeRegressor, BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, SVR, Ridge, GradientBoostingRegressor, Lasso, ElasticNet]

results_df = pd.DataFrame(columns=['Regressor', 'Train_RMSE', 'Test_RMSE', 'Train_R2', 'Test_R2'])

for regressor in regressors:
    regressor_name = regressor.__name__
    train_rmse, test_rmse, train_r2, test_r2 = rmse_and_r2(regressor, X_train_sc, y_train, X_test_sc, y_test)
    results_df = pd.concat([results_df, pd.DataFrame([[regressor_name, train_rmse, test_rmse, train_r2, test_r2]], columns=['Regressor', 'Train_RMSE', 'Test_RMSE', 'Train_R2', 'Test_R2'])], ignore_index=True)

results_df.sort_values(by='Test_R2', ascending=False)

In [ ]:
lr_pipeline = Pipeline([
    ("lr", LinearRegression())
    ])
lr_parameters = {
    'lr__fit_intercept': [True, False],
    'lr__positive': [True, False],
    'lr__copy_X': [True, False]
}
lr_gs = GridSearchCV(lr_pipeline, param_grid=lr_parameters, n_jobs=-1, scoring='neg_root_mean_squared_error')
lr_gs.fit(X_train_sc, y_train)

In [ ]:
knr_pipeline = Pipeline([
    ("knr", KNeighborsRegressor())
    ])
knr_parameters = {
    'knr__n_neighbors': [5, 10, 15, 20, 25, 27, 30, 35],
    'knr__weights': ['uniform', 'distance'],
    'knr__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knr__leaf_size': [5, 10, 15, 20, 25, 27, 30, 35],
    'knr__p' : [1,2]
}
knr_gs = GridSearchCV(knr_pipeline, param_grid=knr_parameters, n_jobs=-1, scoring='neg_root_mean_squared_error')
knr_gs.fit(X_train_sc, y_train)


In [ ]:
dt_pipeline = Pipeline([
    ("dt", DecisionTreeRegressor())
])

dt_parameters = {
    'dt__max_depth': [None, 5, 10, 15, 20],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4]
}

dt_gs = GridSearchCV(dt_pipeline, param_grid=dt_parameters, n_jobs=-1, scoring='neg_root_mean_squared_error')
dt_gs.fit(X_train_sc, y_train)

In [ ]:
br_pipeline = Pipeline([
    ("bagging", BaggingRegressor(estimator=DecisionTreeRegressor()))
])

bagging_parameters = {
    'bagging__n_estimators': [10, 20, 30],
    'bagging__max_samples': [0.5, 0.7, 1.0],
    'bagging__max_features': [0.5, 0.7, 1.0],
    'bagging__bootstrap': [True, False],
    'bagging__estimator__max_depth': [None, 5, 10]
}

bagging_gs = GridSearchCV(br_pipeline, param_grid=bagging_parameters, n_jobs=-1, scoring='neg_root_mean_squared_error')
bagging_gs.fit(X_train_sc, y_train)

In [ ]:
rf_pipeline = Pipeline([
    ("rf", RandomForestRegressor())
])

rf_parameters = {
    'rf__n_estimators': [50, 100, 150],
    'rf__max_depth': [None, 5, 10, 15],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['sqrt', 'log2', None]
}

rf_gs = GridSearchCV(rf_pipeline, param_grid=rf_parameters, n_jobs=-1, scoring='neg_root_mean_squared_error')
rf_gs.fit(X_train_sc, y_train)

In [ ]:
svr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR())
])

svr_parameters = {
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': [1, 10],
    'svr__gamma': ['scale', 'auto'],
    'svr__epsilon': [0.1, 0.5]
}

svr_gs = GridSearchCV(svr_pipeline, param_grid=svr_parameters, n_jobs=-1, scoring='neg_mean_squared_error', cv=5)
svr_gs.fit(X_train_sc, y_train)

In [ ]:
ridge_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])

ridge_parameters = {
    'ridge__alpha': [0.1, 1, 10],
    'ridge__fit_intercept': [True, False],
    'ridge__copy_X': [True, False],
    'ridge__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']
}

ridge_gs = GridSearchCV(ridge_pipeline, param_grid=ridge_parameters, n_jobs=-1, scoring='neg_mean_squared_error')
ridge_gs.fit(X_train_sc, y_train)

In [ ]:
gb_pipeline = Pipeline([
    ("gb", GradientBoostingRegressor())
])

gb_parameters = {
    'gb__n_estimators': [50, 100, 150],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 5, 7],
    'gb__min_samples_split': [2, 5, 10],
    'gb__min_samples_leaf': [1, 2, 4],
    'gb__subsample': [0.8, 1.0]
}

gb_gs = GridSearchCV(gb_pipeline, param_grid=gb_parameters, n_jobs=-1, scoring='neg_mean_squared_error')
gb_gs.fit(X_train_sc, y_train)

In [ ]:
lasso_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso())
])

lasso_parameters = {
    'lasso__alpha': [0.1, 1, 10],
    'lasso__fit_intercept': [True, False],
    'lasso__copy_X': [True, False],
    'lasso__tol': [0.001, 0.01, 0.1]
}

lasso_gs = GridSearchCV(lasso_pipeline, param_grid=lasso_parameters, n_jobs=-1, scoring='neg_mean_squared_error')
lasso_gs.fit(X_train_sc, y_train)

In [ ]:
elasticnet_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("elasticnet", ElasticNet())
])

elasticnet_parameters = {
    'elasticnet__alpha': [0.1, 0.5, 0.9],
    'elasticnet__l1_ratio': [0.4, 0.5, 0.6],
    'elasticnet__fit_intercept': [True, False],
    'elasticnet__copy_X': [True, False],
    'elasticnet__tol': [0.001, 0.01, 0.1]
}

{'elasticnet__alpha': 0.1, 'elasticnet__copy_X': True, 'elasticnet__fit_intercept': False, 'elasticnet__l1_ratio': 0.5, 'elasticnet__tol': 0.01}

elasticnet_gs = GridSearchCV(elasticnet_pipeline, param_grid=elasticnet_parameters, n_jobs=-1, scoring='neg_mean_squared_error')
elasticnet_gs.fit(X_train_sc, y_train)

In [ ]:
lr_results = {
    'Model': 'Linear Regression',
    'Best Score': lr_gs.best_score_ * -1,
    'Best Parameters': lr_gs.best_params_
}

knr_results = {
    'Model': 'KNeighbors Regressor',
    'Best Score': knr_gs.best_score_ * -1,
    'Best Parameters': knr_gs.best_params_
}

dt_results = {
    'Model': 'Decision Tree Regressor',
    'Best Score': dt_gs.best_score_ * -1,
    'Best Parameters': dt_gs.best_params_
}

br_results = {
    'Model': 'Bagging Regressor',
    'Best Score': bagging_gs.best_score_ * -1,
    'Best Parameters': bagging_gs.best_params_
}

rf_results = {
    'Model': 'Random Forest Regressor',
    'Best Score': rf_gs.best_score_ * -1,
    'Best Parameters': rf_gs.best_params_
}

svr_results = {
    'Model': 'Support Vector Regression',
    'Best Score': svr_gs.best_score_ * -1,
    'Best Parameters': svr_gs.best_params_
}

ridge_results = {
    'Model': 'Ridge Regression',
    'Best Score': ridge_gs.best_score_ * -1,
    'Best Parameters': ridge_gs.best_params_
}

gb_results = {
    'Model': 'Gradient Boosting Regressor',
    'Best Score': gb_gs.best_score_ * -1,
    'Best Parameters': gb_gs.best_params_
}

lasso_results = {
    'Model': 'Lasso Regressor',
    'Best Score': lasso_gs.best_score_ * -1,
    'Best Parameters': lasso_gs.best_params_
}

elasticnet_results = {
    'Model': 'Elastic Net Regressor',
    'Best Score': elasticnet_gs.best_score_ * -1,
    'Best Parameters': elasticnet_gs.best_params_
}


results_df2 = pd.DataFrame([lr_results, knr_results, dt_results, br_results, rf_results,
                            svr_results, ridge_results, gb_results, lasso_results, elasticnet_results])

results_df2.sort_values(by='Best Score')

----------------------------------------------------------------------------------------

In [ ]:
y_pred = ridge_gs.predict(X_test_sc)
y_true = y_test
r_squared = r2_score(y_true, y_pred)
print("R-squared:", r_squared)

In [ ]:
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

In [ ]:
mae = mean_absolute_error(y_true, y_pred)
print("Mean Absolute Error:", mae)

In [ ]:
residuals = y_true - y_pred
plt.scatter(y_pred, residuals)
plt.title("Residual Plot")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.show()

In [ ]:
sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution")
plt.show()

In [ ]:
plt.scatter(y_pred, residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title("Residuals vs. Predicted Values")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.show()